# Анализ геолокации фотографий

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib notebook

In [92]:
%config Completer.use_jedi = False

## Сбор данных

vk.com позволяет выгружать список фотографий, сделанных в радиусе r от какой-то точки с помощью метода API photos.search

https://vk.com/dev/photos.search

Для осуществления запростов нужен service token, который достаточно легко получить (https://vk.com/dev/access_token).

In [3]:
import getpass
token = getpass.getpass('Insert service token\n');

Insert service token
········


Запросы посылаем распространенной библиотекой для requests.

Получаемый ответ для удобства сразу конвертируем в JSON.
Получаем что-то вроде такого:

In [18]:
import requests
import json

lat  = 54.5008
long = 33.0088
response = requests.get(
                'https://api.vk.com/method/photos.search?lat={}&long={}&count=1&v=5.130&access_token={}'.\
            format(lat, long, token)
            )
response = json.loads(response.text)

# удалим из ответа ссылки на изображение в разных разрешениях
# ссылки занимают много места и не позволяют с ходу поянть структуру полученных данных 
del response['response']['items'][0]['sizes']

print(json.dumps(response, indent = 1))

{
 "response": {
  "count": 27554,
  "items": [
   {
    "album_id": -7,
    "date": 1615270447,
    "id": 457242496,
    "owner_id": -175649192,
    "has_tags": false,
    "lat": 54.57587,
    "long": 33.184714,
    "text": "",
    "user_id": 100
   }
  ]
 }
}


Видно, что мы можем без проблем получить:

 - координаты фотографии
 - дату и время, когда она была загружена *(или сделана, тут не понятно)*
 - id пользователя, который её заргузил
 - текстовое описание, если пользователь его добавил
 - еще несколько атрибутов, которые нас не сильно инетересуют

### Генерируем точки из которых будем собирать фото

Для сбора данных необходим список интересующих нас точек, в окрестности которых мы и будем искать фотографии.
Пишем функцию, которая генерирует набор точек, лежащих в интересующем нас прямоугольнике.

На вход функции будем подавать координаты *(широта, долгота)* верхней левой и нижней правой вершины прямоугольника и шаг с которым в нём будут располагаться точки.

*Для простоты Землю считаем идеальной сферой.*

In [3]:
def get_points(top_left, bot_right, step):
    step = 2*step
    long1, lat1 = top_left
    long2, lat2 = bot_right
    earth_r = 6371000 #m
  
    if abs(long1-long2)<5:
        long = np.pi/180*(long1+long2)/2
    else:
        raise ValueError(
            'The input area is too big. Difference in longitude is expected to be less than 5 degrees'
            )
        
    one_degree_lat_length = 2*np.pi*earth_r*np.cos(long)/360 #m
    one_degree_long_length = 2*np.pi*earth_r/360 #m
    
    print(one_degree_lat_length)
    
    one_degree_lat_length = 78847 #m
    one_degree_long_length = 110500 #m
    
    lat_step  = step / one_degree_lat_length
    long_step = 1.5*step/np.cos(np.pi/6) / one_degree_long_length
    
    lat = np.arange(lat1, lat2, lat_step)
    long = np.arange(long2, long1, long_step) 
    xx, yy = np.meshgrid(long, lat)
    out_even = np.stack((xx.flatten(),yy.flatten()), axis=-1)
    
    lat = np.arange(lat1+lat_step/2, lat2, lat_step)
    long = np.arange(long2+long_step/2, long1, long_step)
    xx, yy = np.meshgrid(long, lat)
    out_odd = np.stack((xx.flatten(),yy.flatten()), axis=-1)

    out = np.concatenate((out_even, out_odd), axis=0)
        
    print(len(out))
    return out

top_left = (45.0984557078117, 41.87632533877263)
bot_right = (44.96500662165554, 42.08675340577944)
points = get_points(top_left, bot_right, 90)

points = pd.DataFrame(points)
points.columns = ['lat', 'long']
points.to_csv("points.csv")

78583.13010355463
8788


Пример того, как выглядит набор кругов с центрами в сгенерированных точках

![Title](imgs/circles.png)

После того, как мы получили набор точек, можно начинать скачивать всё что мы пожелаем.

![Title](imgs/TP_ready.gif)

Сначала - пишем функцию, скачивающую все фотографии в заданном радиусе от одной точки:

In [ ]:
def get_photos_from_location(long, lat, radius, token, log, error):
    r = requests.get(
                'https://api.vk.com/method/photos.search?long={}&lat={}&count={}&start_time=1230757200&radius={}&v=5.130&access_token={}'.\
            format(lat, long, 0, radius, token)
            ) 
    if 'error' in r.text:
        print('error1')
        error.loc[len(error.index)] = [long,lat,r.text]
        return None
    num_photos = json.loads(r.text)['response']['count']
#     print(num_photos)
    
    
    df = pd.DataFrame()
    count = 1000
    photos_downloaded = 0
    new = pd.DataFrame()

    # костыльный while, который необходим, так как vk выдает только часть фотографий
    # поэтому запоминаем сколько фото у нас уже есть и просим выдать список с соответствующим offset'ом
    # до тех пор пока не скачаем все или пока vk не перестанет нам что-то отдавать
    while photos_downloaded < num_photos:
        r = requests.get(
                'https://api.vk.com/method/photos.search?long={}&lat={}&count={}&offset={}&start_time=1230757200&radius={}&v=5.130&access_token={}'.\
            format(lat, long, count, photos_downloaded, radius, token)
            )
        if 'error' in r.text:
            print('error2')
            error.loc[len(error.index)] = [long,lat,r.text]
            return None
    
        res = json.loads(r.text)
        n = len(res['response']['items'])
        if n == 0:
            break
#         print('n', n)
        new = pd.DataFrame(res['response']['items'])
        new = new.astype({'sizes': 'str'})

        df = df.append(new, ignore_index=True)        
        photos_downloaded += n
    log.loc[len(log.index)] = [long,lat,num_photos,df.shape[0]]
    return df

Функция `get_photos_from_location`, помимо того что собирает информцию о фотографиях в заданном радиусе от нужной точки, еще и логирует ошибки и то, как много фотографий удалось получить.

Затем - функция, которая ходит по массиву точек и собирает все что можно с помощью уже готовой `get_photos_from_location`.

In [ ]:
def get_photos(points, radius, token, loud=True):
    start = time.time()
    
    df = pd.DataFrame({'date':[], 'id':[], 'owner_id':[], 'lat':[], 'long':[], 'sizes':[], 'text':[]})
    log = pd.DataFrame({'long':[],'lat':[],'num_photos':[],'photos_downloaded':[]})
    error = pd.DataFrame({'long':[],'lat':[],'err_msg':[]})
    df.to_csv("/media/drev/DISK/VirtualMachine/data_backup.csv", mode='a')
    log.to_csv("/media/drev/DISK/VirtualMachine/log.csv", mode='a')
    error.to_csv("/media/drev/DISK/VirtualMachine/err.csv", mode='a')
    
    for n, point in enumerate(points):
        long, lat = point
#         print(long, lat)
        data = get_photos_from_location(long, lat, radius, token, log, error)
        if n % 100 == 0:
            if 'post_id' in df.columns:
                df.drop(columns=['post_id'], inplace=True)
            if 'album_id' in df.columns:
                df.drop(columns=['album_id'], inplace=True)
            if 'has_tags' in df.columns:
                df.drop(columns=['has_tags'], inplace=True)
            if 'user_id' in df.columns:
                df.drop(columns=['user_id'], inplace=True)
            
            print('{:.1f}% complete\t{}/{}\t{:.2e} sec passed\t{:.2e} sec left'\
                  .format(n/len(points)*100,n,len(points),time.time()-start,(time.time()-start)/(n+1)*(len(points)-n)))
            df.to_csv("data_backup.csv", mode='a', header=False)
            log.to_csv("log.csv", mode='a', header=False)
            error.to_csv("err.csv", mode='a', header=False)
            
            df = pd.DataFrame()
            log = pd.DataFrame({'long':[],'lat':[],'num_photos':[],'photos_downloaded':[]})
            error = pd.DataFrame({'long':[],'lat':[],'err_msg':[]})
            continue
        if data is None:
            continue
        df = df.append(data, ignore_index=True)
    return None

Так как порой процесс парсинга сильно затягивается, потерять драгоценные биты, полученные в течение часов работы очень не хочется - каждые n шагов сохрняем всё добро в файл. Это помимо прочего позволяет не забивать оперативную память.

Берем в качестве примера замечательный Ставрополь, раскидываем наши круги радиусом 800 метров и смотрим что из этого всего выйдет.

In [ ]:
top_left = (45.0984557078117, 41.87632533877263)
bot_right = (44.96500662165554, 42.08675340577944)
radius = 800

points = get_points(top_left, bot_right, radius)
get_photos(points, radius, token)

Ждем некоторое колличество времени и получаем файлик со всем всей необходимой информацией.

Из любопытства посмотрим на гистограмму распределения числа фотографий в точке:

In [ ]:
sns.histplot()

### Проблема

Из гистограммы видим что из каждой точки, api vk отдает отдает не больше 3000 фотографий. По этой причине мы теряем большую часть данных, что является критической проблемой.

Проблему решаем в лоб: ищем точки из которых удалось выгрузить не всё, даелаем разбивку с меньшим шагом и снова все скачиваем.

In [ ]:
log = pd.read_csv("log.csv")

log['num_photos'] = log['num_photos'].astype(float).astype(int)
log['photos_downloaded'] = log['photos_downloaded'].astype(float).astype(int)

out['num_lost'] = log['num_photos']-log['photos_downloaded']
out = log.loc[log[''num_lost'']>10]

out.sort_values(by='num_lost', ascending=False, inplace=True)
print(out['num_lost'].sum())

one_degree_long_length = 110500 #m
one_degree_lat_length = 2*np.pi*6371000*np.cos(np.pi/180*45.04591029)/360 #m

print(one_degree_lat_length)

fine_points = np.empty((0,2))
print(fine_points)

for i in range(len(out)):
    lat, long = out[i:i+1]['long'].values[0], out[i:i+1]['lat'].values[0]
#     print(long,lat)
    r = 100
    top_left = (long+r/one_degree_long_length, lat-r/one_degree_lat_length)
    bot_right = (long-r/one_degree_long_length, lat+r/one_degree_lat_length)
    fine_points = np.concatenate((fine_points, get_points(top_left, bot_right, 20, dense=False)))
# print(fine_points[:17])
print(fine_points.shape)

Так за несколько итераций, в последней из которых ходили с шагом 10 метров, удалось скачать практически все доступные фотографии с гелокацией из инетересущей области.

Итого получили ХХХ фотографий, что кажется весьма неплохо.

## Чистка данных

Весь огромный массив фотографий полон шума и от которого его необходимо очистить.

Для начала - очевидное: удаляем дубликаты, которые появились из-за пересечения кругов или неточности с которой vk измеряет расстояние

In [ ]:
data1 = pd.read_csv("/media/drev/DISK/VirtualMachine/data_backup_2.csv")
print(data1.shape)

data_clean = data1

data_clean.drop(columns=['sizes','text'], inplace=True)
data_clean = data_clean.drop_duplicates()
data_clean.reset_index(inplace=True)

print(data_clean.shape)

Осталось ХХХ фотографий.

Избавляемся от тех, у которых отсутствуют координаты или дата.
Помимо прочего, удалем залетные фоторгафии, у которых координаты лежат вне указанной нами области:

In [ ]:
index_names = data_clean[data_clean['long'].isna()].index
data_clean.drop(index_names, inplace=True)

index_names = data_clean[data_clean['lat'].isna()].index
data_clean.drop(index_names, inplace=True)

index_names = data_clean[data_clean['date'].isna()].index
data_clean.drop(index_names, inplace=True)

index_names = data_clean[data_clean['long']=='long'].index
data_clean.drop(index_names, inplace=True)

data_clean['long'] = data_clean['long'].astype(float)
data_clean['lat'] = data_clean['lat'].astype(float)

index_names = data_clean[data_clean['long']<top_left[1]].index
data_clean.drop(index_names, inplace=True)
index_names = data_clean[data_clean['long']>bot_right[1]].index
data_clean.drop(index_names, inplace=True)

index_names = data_clean[data_clean['lat']<bot_right[0]].index
data_clean.drop(index_names, inplace=True)
index_names = data_clean[data_clean['lat']>top_left[0]].index
data_clean.drop(index_names, inplace=True)

data_clean['date'] = pd.to_datetime(data_clean['date'], unit='s')

data_clean.to_csv("data_clean.csv")

Чистые данные сохраняем.

Теперь построим все на карте. Для этого хорошо подходит сервис kepler.gl, который сделал Uber для визуализации поездок такси. (ссылка)

Сервис бесплатный, функциональный и без проблем справляется с большими объемами данных

![Гистограмма из kepler](imgs/kepler_hist1.png)

Вот такая красота у нас получилась.

Бросаются в глаза аномальные высоченные пики, расположенные далеко не в центре города. После внимательного изучения стало ясно, что эти пики соответствуют положению особо активных пользователей, которые публикуют оочень много фотографий. Как правило это онлайн-магазины которые загружают пачки фотографий продаваемых товаров.

### Удаляем посты от отдельных юзеров

In [ ]:
n_bins = 200
data['lat_bin'] = pd.qcut(data['lat'], n_bins, labels=False)
data['long_bin'] = pd.qcut(data['long'], n_bins, labels=False)

data['feature'] = data['owner_id'].astype(str) + '_' + data['long_bin'].astype(str) + '_' + data['long_bin'].astype(str)

data_filtered = data.loc[~data['feature'].duplicated()]

## Анализ данных

### Ищем интересные места

### Ищем особенности в данных в зависимости от даты/ времени суток/ времени года